In [1]:
from stratopy import metadatatools
from stratopy.extractors import cloudsat
from stratopy.extractors import goes
from stratopy.transformers import mergers
from stratopy.transformers import scalers

Download 1 band file from ABI GOES16

In [2]:
goes_obj = goes.GOES16(product_type="ABI-L2-CMIPF", channel=13)
goes_data = goes_obj.fetch("2019 jan 2 19:30")

In [13]:
goes_data.data

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                       datetime64[ns] ...
  * y                                       (y) float64 0.1518 ... -0.1518
  * x                                       (x) float64 -0.1518 ... 0.1518
    y_image                                 float32 ...
    x_image                                 float32 ...
    band_wavelength                         (band) float32 ...
    band_id                                 (band) int32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (y, x) float32 ...
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 ...
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 ...
    percent_uncorrectable_GRB_errors        float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    earth_sun_distance_anomaly_in_AU        float32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/29)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    cdm_data_type:             Image
    time_coverage_start:       2019-01-02T19:00:36.3Z
    time_coverage_end:         2019-01-02T19:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        5c74601f-92ef-4f97-ae10-919b95261557

Lets see one transformer. This one normalizes an ABI image to [0,1].

Returns an xarray with an image of shape (n_bands, image_width, image_height).

In [3]:
norm_goes = scalers.MinMaxNormalize().transform(goes_data)
norm_goes

AttributeError: module 'stratopy.metadatatools' has no attribute 'instrument_type'

Download a file from CloudSat.

In [14]:
from unittest import mock

with mock.patch("paramiko.RSAKey.from_private_key_file", return_value="pkey"):
    with mock.patch("paramiko.SSHClient.connect"):
        cs_obj = cloudsat.CloudSat(
        product_type="2B-CLDCLASS.P1_R05",
        username="fakeusr@gmail.edu",
        keyfile="some_file",
        )
        
        csat_data = cs_obj.fetch("2019 dec 12 15:00")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/pola/.virtualenvs/stratopy/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2311/694148380.py", line 11, in <module>
    csat_data = cs_obj.fetch("2019 dec 12 15:00")
  File "/home/pola/.virtualenvs/stratopy/StratoPy/stratopy/extractors/ebase.py", line 204, in fetch
    fp = from_cache(
  File "/home/pola/.virtualenvs/stratopy/StratoPy/stratopy/utils/cache.py", line 97, in from_cache
    value = function(*args, **kwargs)
  File "/home/pola/.virtualenvs/stratopy/StratoPy/stratopy/extractors/cloudsat.py", line 344, in _download
  File "/home/pola/.virtualenvs/stratopy/lib/python3.9/site-packages/paramiko/client.py", line 573, in open_sftp
    return self._transport.open_sftp_client()
AttributeError: 'NoneType' object has no attribute 'open_sftp_client'

During handling of the above exception, another exception occurred:

Traceb

In [ ]:
CLOUDSAT_PATH = "../../../tests/data/CloudSat/2019002175851_67551_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf"
csat_data = cloudsat.read_hdf4(CLOUDSAT_PATH)
csat_data = metadatatools.add_metadata(
        csat_data,
        orbit_type=metadatatools.POLAR,
        platform=metadatatools.CLOUDSAT,
        instrument_type=metadatatools.RADARS,
        product_key="2B-CLDCLASS.P1_R05",
    )

Merge products

In [ ]:
merged_data = mergers.MergePolarGeos("2019 2 jan 18:30", 
                                     time_zone="UTC", 
                                     trim_size=(3,3)).transform(csat_data, norm_goes)

In [ ]:
merged_data

<xarray.Dataset>
Dimensions:           (cloudsat_trace: 36950, z: 125, layer: 10, nbands: 1,
                       img_wide: 3, img_height: 3)
Coordinates:
    trace             object <built-in method copy of numpy.ndarray object at...
    height            (cloudsat_trace, z) int16 -9999 -9999 ... -9999 -9999
    layers            object <built-in method copy of numpy.ndarray object at...
    lat               (cloudsat_trace) float64 -0.009365 -0.01907 ... 0.004176
    lon               (cloudsat_trace) float64 117.1 117.1 117.1 ... 92.5 92.5
    time              (cloudsat_trace) datetime64[ns] 2019-01-02T17:59:01.740...
    precip_flag       (cloudsat_trace) int8 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9
    land_sea_flag     (cloudsat_trace) float32 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0
  * cloudsat_trace    (cloudsat_trace) int32 0 1 2 3 ... 36946 36947 36948 36949
  * nbands            (nbands) int8 1
  * img_wide          (img_wide) int8 1 2 3
  * img_height        (img_height) int8 1 2 3
Dimensions without coordinates: z, layer
Data variables:
    cloud_scenario    (cloudsat_trace, z) int16 2081 2081 2081 ... 2113 2113
    cloud_layer_type  (cloudsat_trace, layer) int8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0
    cloud_layer_base  (cloudsat_trace, layer) float32 -99.0 -99.0 ... -99.0
    cloud_layer_top   (cloudsat_trace, layer) float32 -99.0 -99.0 ... -99.0
    geos              (cloudsat_trace, nbands, img_wide, img_height) float32 ...
Attributes:
    TAIstart:             820605541.74
    UTCstart:             [[64731.73828125]]
    bin_size:             [239.82907104]
    time_coverage_start:  2019-01-02T17:59:01.740000+00:00
    time_coverage_end:    2019-01-02T19:37:33.579844+00:00
    platform_ID:          CloudSat
    _STRATOPY_:           Metadata(orbit_type='polar', platform='CloudSat', i...

Nueva version

In [4]:
#goes_data.time_coverage_start,goes_data.time_coverage_end
pat = "../../../tests/data/GOES16/OR_ABI-L2-CMIPF-M3C13_G16_s20190040600363_e20190040611141_c20190040611220.nc"
import xarray as xa
gdata = xa.open_dataset(pat, engine="h5netcdf")


In [ ]:
merged_data.time_coverage_start, merged_data.time_coverage_end

('2019-01-02T17:59:01.740000+00:00', '2019-01-02T19:37:33.579844+00:00')

In [ ]:
goes_data.attrs

{'naming_authority': 'gov.nesdis.noaa',
 'Conventions': 'CF-1.7',
 'Metadata_Conventions': 'Unidata Dataset Discovery v1.0',
 'standard_name_vocabulary': 'CF Standard Name Table (v25, 05 July 2013)',
 'institution': 'DOC/NOAA/NESDIS > U.S. Department of Commerce, National Oceanic and Atmospheric Administration, National Environmental Satellite, Data, and Information Services',
 'project': 'GOES',
 'production_site': 'NSOF',
 'production_environment': 'OE',
 'spatial_resolution': '2km at nadir',
 'orbital_slot': 'GOES-East',
 'platform_ID': 'G16',
 'instrument_type': 'GOES R Series Advanced Baseline Imager',
 'scene_id': 'Full Disk',
 'instrument_ID': 'FM1',
 'dataset_name': 'OR_ABI-L2-CMIPF-M3C13_G16_s20190021800363_e20190021811141_c20190021811221.nc',
 'iso_series_metadata_id': '8c9e8150-3692-11e3-aa6e-0800200c9a66',
 'title': 'ABI L2 Cloud and Moisture Imagery',
 'summary': 'Single emissive band Cloud and Moisture Imagery Products are digital maps of clouds, moisture and atmospheric 

In [10]:
data = metadatatools.SatelliteData(gdata, "a", "Radar", "GOES", 'Geostationary',  "27 jun 22 12:03", "27 jun 22 12:13") 
da = data.data
da

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                       datetime64[ns] ...
  * y                                       (y) float64 0.1518 ... -0.1518
  * x                                       (x) float64 -0.1518 ... 0.1518
    y_image                                 float32 ...
    x_image                                 float32 ...
    band_wavelength                         (band) float32 ...
    band_id                                 (band) int32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (y, x) float32 ...
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 ...
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 ...
    percent_uncorrectable_GRB_errors        float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    earth_sun_distance_anomaly_in_AU        float32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/29)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    cdm_data_type:             Image
    time_coverage_start:       2019-01-04T06:00:36.3Z
    time_coverage_end:         2019-01-04T06:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        7132a0ee-fb9e-4647-8a2e-11abc9589a63

In [11]:
data

<SatelliteData product_key='a', instrument_type='Radar', platform='GOES', orbit_type='Geostationary', time_start='27 jun 22 12:03', time_end='27 jun 22 12:13'>